# NEO DATA MANAGER

In [257]:
import requests 
import json 
import pandas as pd
from pandas.io.json import json_normalize

API_URL = 'https://api.nasa.gov/neo/rest/v1/feed'
API_PARAMS = '?start_date={}&end_date={}&api_key={}'

#NASA only stores most recent 10 days of data
start_date = '2019-09-01'
end_date = '2019-09-07'

api_url = API_URL + API_PARAMS.format(start_date,end_date,'KwRDL3x7w75naVBiJua5GmSpMEDn75yNMjMqacS8')

response=requests.get(api_url)
response.raise_for_status()
neodata=json.loads(response.text)

In [271]:
from flask import Flask
app = Flask(__name__)

@app.route("/output")
def output():
    return "Hello World!"

# Approach 1

#### It is much faster than approach 2 but requires modification to access close approach data

In [258]:
program_starts = time.time()
new_df = pd.concat([pd.DataFrame(json_normalize(neodata['near_earth_objects'],x,max_level=10)) for x in neodata['near_earth_objects'].keys()]).reset_index(drop=True)
now = time.time()
print("It has been {0} seconds since the loop started".format(now - program_starts))


print(new_df.shape)
#print(json.dumps(neodata, sort_keys=True, indent=4))
new_df.head(2)

It has been 0.04986739158630371 seconds since the loop started
(86, 17)


,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,3843599,3843599,(2019 QL1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843599,22.900,False,"[{'close_approach_date': '2019-09-05', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3843599?ap...,0.069913,0.156329,69.912523,156.329154,0.043442,0.097138,229.371803,512.890943
1,3843554,3843554,(2019 PA3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843554,21.196,False,"[{'close_approach_date': '2019-09-05', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3843554?ap...,0.153234,0.342642,153.233943,342.641513,0.095215,0.212907,502.736049,1124.151981


# Approach 2

##### Is able to access all the required data but is much slower than Approach 2

In [266]:
print("Today " + str(neodata["element_count"]) + " asteroids will be passing close to planet Earth:")
print("")
asteroids = neodata["near_earth_objects"]
print(asteroids.keys())

tmp = pd.DataFrame()
program_starts = time.time()
for asteroid in asteroids:
    for field in asteroids[asteroid]:
            tmp = tmp.append({'Estimated_Diameter': round((field["estimated_diameter"]["meters"]["estimated_diameter_min"]+field["estimated_diameter"]["meters"]["estimated_diameter_max"])/2),
                            'Close_Approach_Date_Time': field["close_approach_data"][0]["close_approach_date_full"],
                            'Velocity':field["close_approach_data"][0]["relative_velocity"]["kilometers_per_hour"],
                            'Distance_to_Earth':field["close_approach_data"][0]["miss_distance"]["kilometers"], 
                            'Asteroid_name':field["name"],
                            'Hazardous':field["is_potentially_hazardous_asteroid"],
                            'neo_ref_id':field["id"]},ignore_index=True)
now = time.time()
print("It has been {0} seconds since the loop started".format(now - program_starts))
tmp.head()
#No asteroids are repeated 
tmp['neo_ref_id'].is_unique

Today 86 asteroids will be passing close to planet Earth:

dict_keys(['2019-09-05', '2019-09-06', '2019-09-03', '2019-09-04', '2019-09-07', '2019-09-01', '2019-09-02'])
It has been 0.3560497760772705 seconds since the loop started


True